In [ ]:
%pylab inline
import seaborn as sns
import numpy as np

### Moving toward a join action, we can use the map() method to create Key/Value pairs from the tuple

### Here we create an RDD from a csv stored in s3, and use the collect() action, which returns an array


In [ ]:
usersWithGenderJoinedRDD = usersRDD.join(gendersRDD)
usersWithGenderJoinedRDD.collect()

# In this notebook, we'll explore a few methods in Spark Core
### Creating RDDs, collect(), map(), join(), reduceByKey(), aggregateByKey(), and filter()

### The collect action causes data to flow across the network from the worker nodes to the master (where you are running the jupyter notebook, or your data analysis)


In [ ]:
#a csv of id, users
rawUsersRDD = sc.textFile("s3n://insight-spark-after-dark/users-sm.csv")
rawUsersRDD.collect()

In [ ]:
def rec_tup(record):
    tokens = record.split(",")
    return (int(tokens[0]), str(tokens[1]))

### We'll start with an online-dating dataset, described here: https://sites.google.com/a/insightdatascience.com/spark-lab/s3-data/dating-profiles

In [ ]:
usersRDD = rawUsersRDD.map(rec_tup)
usersRDD.collect()

# Next Steps

Here are some further questions to get to grips with Spark basics

In [ ]:
#a csv of id, genders
rawGendersRDD = sc.textFile("s3n://insight-spark-after-dark/gender-sm.csv")
rawGendersRDD.collect()

### Task 1: There exists another zipped csv, s3n://insight-spark-after-dark/gender.csv.gz. Create an RDD called genders_raw

In [ ]:
# Task 1: There exists another zipped csv, s3n://insight-spark-after-dark/gender.csv.gz.
# Create an RDD called genders_raw and read in the full gender file (UserId, gender).
genders_raw = sc.textFile("s3n://insight-spark-after-dark/gender.csv.gz")
genders_raw.take(5)

In [ ]:
gendersRDD = rawGendersRDD.map(rec_tup)
gendersRDD.collect()

In [ ]:
# Task 2: Parse each of the record  from the genderRDDD so that we have a tuple of (gender(string), id(int)) and show 
split_rdd = genders_raw.map(lambda r: r.split(','))
parsed_rdd = split_rdd.map(lambda r: (r[1], int(r[0])))
parsed_rdd.take(5)

### Task 3: Calculate the distribution of Males, Females, and Unknowns and show the results


### Now that we have two RDDs with Key/Value pairs, use the join method to join the RDDs based on the Key

In [ ]:
x = np.arange(3)
y = np.array(count)
sns.barplot(x,y)


In [ ]:
# Task 3: Calculate the distribution of Males, Females, and Unknowns and show the results
tagged_rdd = parsed_rdd.map(lambda r: (r[0], 1))
distribution = tagged_rdd.reduceByKey(lambda x,y: x+y)
distr = distribution.collect()
gender = [str(val[0]) for val in distr]
count = [val[1] for val in distr]

print gender
print count

### Task 4: Calculate the sum of all the IDs for each gender


### Task 2: Parse each of the record  from the genderRDDD so that we have a tuple of (gender(string), id(int)) and show 


In [ ]:
# Task 4: Calculate the sum of all the IDs for each gender
id_sum_rdd = parsed_rdd.reduceByKey(lambda x,y: x+y)
id_sum_rdd.collect()

### Task 5: Calculate the average of all the IDs for each gender


In [ ]:
# Task 5: Calculate the average of all the IDs for each gender
sum_cnt_rdd = parsed_rdd.aggregateByKey((0,0),
                                       lambda x,y: (x[0]+y, x[1]+1),
                                       lambda x,y: (x[0]+y[0], x[1]+y[1]))
avg_rdd = sum_cnt_rdd.mapValues(lambda r: r[0]/r[1])
avg_rdd.collect()

### Task 6: Only return records from parsed_rdd which have ids that are a multiple of 5 and show the first 5 results


In [ ]:
# Task 6: Only return records from parsed_rdd which have ids that are a multiple of 5 and 
# show the first 5 results
filtered_rdd = parsed_rdd.filter(lambda r: r[1]%5==0)
filtered_rdd.take(5)